# Import Libraries


In [1]:
import numpy as np
import pandas as pd

from seglearn.pipe import Pype
from seglearn.transform import Interp, Segment, patch_sampler, FeatureRep
from seglearn.feature_functions import base_features, all_features, hudgins_features, emg_features
from seglearn.datasets import load_watch

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, FunctionTransformer, PolynomialFeatures
from sklearn.model_selection import GridSearchCV, GroupKFold, cross_validate
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

import matplotlib.pyplot as plt

import random
import itertools

from joblib import dump, load
#%matplotlib notebook

# Importand paths

In [2]:
graphs_path = "./Graphs/"

## Loading the Data

In [3]:
X = np.load("./Data/Preprocessed_Data/data.npy", allow_pickle=True)
y = np.load("./Data/Preprocessed_Data/labels.npy", allow_pickle=True)
groups = np.load("./Data/Preprocessed_Data/groups.npy", allow_pickle=True)

# Converting them to list (needed for the learning mechanism)
X = X.tolist()
y = y.tolist()
groups = groups.tolist()

## Loading data from other dataset

In [4]:
#data = load_watch()
#X = data['X']
#y = data['y']
#groups = data["subject"]

# Defining the features and the pipeline

In [5]:
sampling_strategy={115 : 300, 104 : 250, 102 : 250, 107 : 250, 101 : 250, 103 : 250, 106 : 150, 105 : 123}

In [6]:
sampling_frequency = 10.
def add_period(X):
    """
    I am adding in a column to represent time (10 Hz sampling), since my data doesn't include it
    the Interp class assumes time is the first column in the series.
    """
    return np.array([np.column_stack([np.arange(len(X[i])) / sampling_frequency, X[i]]) for i in np.arange(len(X))])

In [7]:
pipeline = Pype([
    #("timer", FunctionTransformer(add_period)),
    #("interp", Interp(sample_period = 1. / sampling_frequency, categorical_target=True)),
    ("segment", Segment(overlap=0.2, shuffle=True)),
    #("resampler", patch_sampler(RandomUnderSampler)(sampling_strategy="all")),
    ("features", FeatureRep(features = {**base_features(), **emg_features()})),
    ("scaler", StandardScaler()),
    ("pca", PCA()),
    #("rf", RandomForestClassifier(criterion="gini"))
    ("lr",  LogisticRegression(multi_class='multinomial', max_iter=1000))

], memory=None)

/home/geo/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass memory=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


## Hyperparameter tuning

In [8]:
splitter = GroupKFold(n_splits=7)
cv = splitter.split(X, y, groups)

parameters_grid = {#"interp__sample_period": [0.2], # 0.1
                   "segment__width": [50],
                   #"pca__n_components" : [60],
                   #"rf__n_estimators": [50],
                   #"rf__min_samples_split": [6],
                  }


# scoring does not work for some reason
# it maybe always say the big category for each fragment
# scoring="accuracy", maybe because random forest has out of bag error 
grid_search = GridSearchCV(pipeline, parameters_grid, cv=cv, n_jobs=-1)
grid_search.fit(X, y)

print("GridSearchCV best clf score: %.2f" % (grid_search.best_score_))
print(grid_search.best_estimator_)

/home/geo/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass memory=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/home/geo/.local/lib/python3.6/site-packages/seglearn/transform.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  for i in np.arange(N)])


GridSearchCV best clf score: 0.66
Pype(steps=[('segment', Segment(overlap=0.2, shuffle=True, width=50)),
            ('features',
             FeatureRep(features={'abs_energy': <function abs_energy at 0x7fe1a8e77d08>,
                                  'emg_var': <function emg_var at 0x7fe1a8e6dbf8>,
                                  'integrated_emg': <function abs_sum at 0x7fe1a8e77c80>,
                                  'kurt': <function kurt at 0x7fe1a8e6d1e0>,
                                  'max': <function maximum at 0x7fe1a8e6d0d0>,
                                  'mean': <function mean at 0x7fe...
                                  'slope_sign_changes': slope_sign_changes(threshold=0),
                                  'std': <function std at 0x7fe1a8e77d90>,
                                  'var': <function var at 0x7fe1a8e77e18>,
                                  'waveform_length': <function waveform_length at 0x7fe1a8e6d730>,
                                  'willison_a

## Cross validation on the best model

In [ ]:
best_estimator = grid_search.best_estimator_

cv = splitter.split(X, y, groups)
#scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
cv_scores = cross_validate(best_estimator, X, y, cv=cv, n_jobs=-1)

cv_scores_df = pd.DataFrame(cv_scores)
cv_scores_df["test_score"]

/home/geo/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass memory=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [ ]:
def plot_confusion_matrix(cm, classes, fold, normalize=True, cmap=plt.cm.Blues):
    title = "Confusion matrix " + str(fold)
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
        
    plt.figure(figsize = (15, 8))
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    
    plt.title(title, fontsize=15)
    plt.ylabel("True label", fontsize=15)
    plt.xlabel("Predicted label", fontsize=15)
    plt.tight_layout()
    plt.savefig(graphs_path + title, facecolor = "#E0E0E0")
    plt.show()

## Creating the confusion matrix for each fold

In [ ]:
df_activities = pd.read_csv("./Data/HomoreDataFromVariousActivities/activities.csv")

cv = splitter.split(X, y, groups)
for fold, split in enumerate(cv):
    training_set, test_set = split
    
    # Spliting the to training and testing set
    X_train, X_test = np.asarray(X)[training_set], np.asarray(X)[test_set]
    y_train, y_test = np.asarray(y)[training_set], np.asarray(y)[test_set]
    
    visited = dict()
    labels = list()
    for yi_label in y_test:
        if yi_label not in visited:
            visited[yi_label] = True
            labels.append(yi_label)

    # Fiting the model
    best_estimator = best_estimator.fit(X_train, y_train)
    
    # Predicting with trained model
    # Transform_predict generates true and predicted target values for the segments
    y_true, y_pred = best_estimator.transform_predict(X_test, y_test)
    
    # Printing accuracy score and confusion matrix
    confusion_mat = confusion_matrix(y_true, y_pred, labels=labels)
    #confusion_mat = confusion_matrix(y_true, y_pred)
    clf_accuracy = accuracy_score(y_true, y_pred)
    
    # Select the right labels and order them correcly
    labels_names = df_activities.loc[df_activities["ACTIVITY_ID"].isin(labels)]
    labels_names = labels_names.iloc[pd.Categorical(labels_names["ACTIVITY_ID"], categories = labels, ordered=True).argsort()]
    labels_names = labels_names["NAME"].to_list()
    plot_confusion_matrix(confusion_mat, labels_names, fold, normalize=True)
    
    print("Accuracy = %.2f" % (clf_accuracy))
    #print(confusion_mat)
    

# Dump and Load the model

In [ ]:
# Dump the model
best_estimator = best_estimator.fit(X, y)
model_path = './Model/classifier.joblib'
dump(best_estimator, model_path) 

In [ ]:
# Load the model
clf = load(model_path)
dummy_data = np.random.rand(150, 6)
dummy_data = [dummy_data]
pred = clf.predict(dummy_data)
pred